# Chisinau Commute Upgrade — Corridor Analysis
**Airport ↔ Botanica | Baseline vs Adaptive Signal Control**

Run order:
1. `python src/network/extract_osm.py`
2. `python src/network/build_sumo_net.py`
3. `python src/demand/od_matrix.py`
4. `python src/demand/trip_generation.py`
5. `python src/demand/signals.py`
6. `python src/simulation/run_sumo.py`
7. `python src/simulation/metrics.py`
8. This notebook

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from analysis.plots import set_style, travel_time_cdf, queue_heatmap, bar_bottleneck_rank

set_style()
%matplotlib inline
plt.rcParams['figure.dpi'] = 120

CONFIG_PATH = 'src/config/scenario.yaml'
with open(CONFIG_PATH) as f:
    cfg = yaml.safe_load(f)

OUT = cfg['outputs']['dir']
print('Config loaded. Output dir:', OUT)

## 1. Load simulation outputs

In [ ]:
trip_df = pd.read_parquet(f'{OUT}/baseline_metrics.parquet')
edge_df = pd.read_parquet(f'{OUT}/edgedata_parsed.parquet')
bn_df   = pd.read_csv(f'{OUT}/baseline_bottlenecks.csv')

with open(f'{OUT}/corridor_kpis.yaml') as f:
    kpis = yaml.safe_load(f)

print(f'Vehicles: {len(trip_df):,}   |  Edge-intervals: {len(edge_df):,}')
trip_df.head(3)

## 2. Corridor KPI summary

In [ ]:
kpi_display = pd.Series(kpis).rename('Value').to_frame()
kpi_display

## 3. Travel time distribution

In [ ]:
completed = trip_df[trip_df['completed']]
peaks = cfg['simulation']['peak_hours']

series_dict = {'All day': completed['travel_time_min']}
for pk_name, pk in peaks.items():
    mask = (completed['depart'] >= pk['start']) & (completed['depart'] < pk['end'])
    series_dict[pk_name] = completed[mask]['travel_time_min']

fig, ax = plt.subplots(figsize=(10, 5))
travel_time_cdf(series_dict, ax=ax, title='Travel Time CDF by Period')
plt.tight_layout()

## 4. Congestion heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(16, 7))
queue_heatmap(edge_df, value_col='speed', top_n=20, ax=ax,
              title='Speed Heatmap — Top 20 Congested Edges (m/s)')
plt.tight_layout()

## 5. Bottleneck ranking

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
bar_bottleneck_rank(bn_df.head(10), ax=ax)
plt.tight_layout()
bn_df.head(10)

## 6. Before / After comparison
_Run adaptive simulation first, then load comparison data_

In [ ]:
ADAPTIVE_OUT = f'{OUT}/adaptive'
adaptive_metrics = f'{ADAPTIVE_OUT}/baseline_metrics.parquet'

if os.path.exists(adaptive_metrics):
    trip_adaptive = pd.read_parquet(adaptive_metrics)
    series_cmp = {
        'Baseline (fixed)': completed['travel_time_min'],
        'Adaptive (pressure)': trip_adaptive[trip_adaptive['completed']]['travel_time_min'],
    }
    fig, ax = plt.subplots(figsize=(10, 5))
    travel_time_cdf(series_cmp, ax=ax, title='Before vs After — Travel Time CDF')
    plt.tight_layout()
else:
    print('Adaptive run not found. Run: python src/simulation/run_sumo.py --mode traci')